### Import all the libraries 

In [1]:
import cv2
from pathlib import Path
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import io
import tensorflow as tf
import numpy as np
import matplotlib
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image as kimg


### Model Architecture

In [2]:
model = Sequential()
model.add(Conv2D(32, (5,5), padding='same', activation='relu',input_shape=(200, 200, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (5,5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (5,5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (5,5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (5,5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (5,5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 64)        5

In [3]:
model.load_weights("CNN_equHeat/cp-050.pkl")

In [4]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
# import cv2
# from pathlib import Path
# import glob
# import numpy as np
# import matplotlib.pyplot as plt
# from PIL import Image
# count = 0
# path=Path('/home/jhanvipatel/Desktop/python/Skin/')
# print(path)
# images=[]
# for img in glob.glob("normal/*.jpg"):
#     print("img:",img)
#     img = kimg.load_img(img,target_size=(200,200))
#     images = kimg.img_to_array(img)
#     # expand dimension of image
#     gray_img=np.expand_dims(images,axis=0)
#     ans = model.predict(gray_img)
#     print("Model Prediction:",ans)
#     count = count + 1
# # print("finalcountofimages:",count)

/home/jhanvipatel/Desktop/python/Skin
img: normal/p9.jpg
Model Prediction: [[1. 0. 0.]]
img: normal/p8.jpg
Model Prediction: [[1. 0. 0.]]
img: normal/p7.jpg
Model Prediction: [[0. 0. 1.]]
img: normal/p4.jpg
Model Prediction: [[0. 0. 1.]]
img: normal/p6.jpg
Model Prediction: [[0. 0. 1.]]
img: normal/p10.jpg
Model Prediction: [[1. 0. 0.]]
img: normal/p3.jpg
Model Prediction: [[1.1339002e-16 0.0000000e+00 1.0000000e+00]]
img: normal/p2.jpg
Model Prediction: [[0. 0. 1.]]
img: normal/p1.jpg
Model Prediction: [[1. 0. 0.]]
img: normal/p5.jpg
Model Prediction: [[1. 0. 0.]]


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import cv2
import argparse
import glob
import os

# def read_this(image_file, gray_scale=False):
#     image_src = cv2.imread(image_file)
#     if gray_scale:
#         image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
#     else:
#         image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
#     return image_src

def enhance_contrast(image_matrix, bins=256):
    image_flattened = image_matrix.flatten()
    image_hist = np.zeros(bins)

    # frequency count of each pixel
    for pix in image_matrix:
        image_hist[pix] += 1

    # cummulative sum
    cum_sum = np.cumsum(image_hist)
    norm = (cum_sum - cum_sum.min()) * 255
    # normalization of the pixel values
    n_ = cum_sum.max() - cum_sum.min()
    uniform_norm = norm / n_
    uniform_norm = uniform_norm.astype(np.uint8)

    # flat histogram
    image_eq = uniform_norm[image_flattened]
    # reshaping the flattened matrix to its original shape
    image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)

    return image_eq

def equalize_this(image_file, with_plot=False, gray_scale=False, bins=256):
#     image_src = read_this(image_file=image_file, gray_scale=gray_scale)
    gray_image = cv2.cvtColor(image_file, cv2.COLOR_BGR2GRAY)
#     if not gray_scale:
#         r_image = image_src[:, :, 0]
#         g_image = image_src[:, :, 1]
#         b_image = image_src[:, :, 2]

#         r_image_eq = enhance_contrast(image_matrix=r_image)
#         g_image_eq = enhance_contrast(image_matrix=g_image)
#         b_image_eq = enhance_contrast(image_matrix=b_image)

#         image_eq = np.dstack(tup=(r_image_eq, g_image_eq, b_image_eq))
#         cmap_val = None
#         image_eq = image_eq.astype(np.uint8)
#         #         cv2.imwrite(img, image_eq)
# #         io.imsave(img, image_eq)             #save the equilised image by replacing the original image.

#     else:
    image_eq = enhance_contrast(image_matrix=gray_image)
    cmap_val = 'gray'
    #         #cv2.imwrite(img, image_eq)
#         io.imsave(img, image_eq)             #save the equilised image by replacing the original image.
    return image_eq



path=Path('/home/jhanvipatel/Desktop/python/Skin/')
print(path)
images=[]
# for imagepath in glob.glob("new_folder/*.jpg"):
#     print("img:",imagepath)
#     image = cv2.imread(imagepath,0)
#     heatmap2 = cv2.applyColorMap(image, cv2.COLORMAP_HOT)
#     print(type(heatmap2))
# #     img_resize = cv2.resize(heatmap2,(200,200))
# #     print(img_resize.shape)
# #     dim = (200, 200)
 
# #     # resize image
# #     resized = cv2.resize(heatmap2, dim, interpolation = cv2.INTER_AREA)
 
# #     print('Resized Dimensions : ',resized.shape)
#     cv2.imwrite('heatmapimg.jpg',heatmap2)
#     equ_img = equalize_this(heatmap2, with_plot=False,gray_scale=True )
#     cv2.imwrite('equimg.jpg',equ_img)
#     images = kimg.img_to_array(equ_img)
#     gray_img=np.expand_dims(images,axis=0)
#     ans = model.predict(gray_img)
#     ans = np.argmax(ans)
#     print("Model Prediction:",ans)
    
    
# for imagepath in glob.glob("new_folder/*.jpg"):
#     print("img:",imagepath)
#     image = cv2.imread(imagepath, 0)
#     heatmap2 = cv2.applyColorMap(image, cv2.COLORMAP_HOT)
#     heatmapimg = cv2.imwrite(imagepath, heatmap2)
# time.sleep(5)
# for img in glob.glob("new_folder/*.jpg"):
#     print("img_name:",img)
#     imArray = cv2.imread(img)
#     equalize_this(image_file=img, with_plot=False,gray_scale=True )

# count = 0
# for img in glob.glob("new_folder/*.jpg"):
#     print("img:",img)
#     img = kimg.load_img(img,target_size=(200,200))
#     images = kimg.img_to_array(img)
#     # expand dimension of image
#     gray_img=np.expand_dims(images,axis=0)
#     ans = model.predict(gray_img)
#     print("Model Prediction:",ans)
#     count = count + 1    

imagepath = input("enter image path:")
image = cv2.imread(imagepath, 0)
heatmap2 = cv2.applyColorMap(image, cv2.COLORMAP_HOT)
# heatmapimg = cv2.imwrite(imagepath, heatmap2)
time.sleep(3)
# imArray = cv2.imread(imagepath)
equalize_this(image_file=heatmap2, with_plot=False,gray_scale=True )
time.sleep(3)
img = kimg.load_img(imagepath,target_size=(200,200))
images = kimg.img_to_array(img)
# expand dimension of image
gray_img=np.expand_dims(images,axis=0)
ans = model.predict(gray_img)
print("Model Prediction:",ans)

/home/jhanvipatel/Desktop/python/Skin
enter image path:/home/jhanvipatel/Desktop/python/Skin/p2.jpg
Model Prediction: [[0. 1. 0.]]


In [54]:
import cv2
from pathlib import Path
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import io
import tensorflow as tf
import numpy as np
import matplotlib
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image as kimg

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

import cv2
from pathlib import Path
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def model_architecture():
    model = Sequential()
    model.add(Conv2D(32, (5,5), padding='same', activation='relu',input_shape=(200, 200, 3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, (5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    model.load_weights("CNN_equHeat/cp-050.pkl")
    
    return model

   

def mymodel(imagefilename,model):
    def read_this(image_file, gray_scale=False):
        image_src = cv2.imread(image_file)
        if gray_scale:
            image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
        else:
            image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
        return image_src

    def enhance_contrast(image_matrix, bins=256):
        image_flattened = image_matrix.flatten()
        image_hist = np.zeros(bins)

        # frequency count of each pixel
        for pix in image_matrix:
            image_hist[pix] += 1

        # cummulative sum
        cum_sum = np.cumsum(image_hist)
        norm = (cum_sum - cum_sum.min()) * 255
        # normalization of the pixel values
        n_ = cum_sum.max() - cum_sum.min()
        uniform_norm = norm / n_
        uniform_norm = uniform_norm.astype(np.uint8)

        # flat histogram
        image_eq = uniform_norm[image_flattened]
        # reshaping the flattened matrix to its original shape
        image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)

        return image_eq

    def equalize_this(image_src, with_plot=False, gray_scale=False, bins=256):
    #     image_src = read_this(image_file=image_file, gray_scale=gray_scale)
        if not gray_scale:
            r_image = image_src[:, :, 0]
            g_image = image_src[:, :, 1]
            b_image = image_src[:, :, 2]

            r_image_eq = enhance_contrast(image_matrix=r_image)
            g_image_eq = enhance_contrast(image_matrix=g_image)
            b_image_eq = enhance_contrast(image_matrix=b_image)

            image_eq = np.dstack(tup=(r_image_eq, g_image_eq, b_image_eq))
            cmap_val = None
            image_eq = image_eq.astype(np.uint8)
            #         cv2.imwrite(img, image_eq)
            #io.imsave(img, image_eq)             #save the equilised image by replacing the original image.

        else:
            image_eq = enhance_contrast(image_matrix=image_src)
            cmap_val = 'gray'
            #         cv2.imwrite(img, image_eq)
            #io.imsave(img, image_eq)             #save the equilised image by replacing the original image.
        return image_eq
    
    image = cv2.imread(imagefilename, 0)
    heatmap2 = cv2.applyColorMap(image, cv2.COLORMAP_HOT)
    equ_img = equalize_this(heatmap2, with_plot=False,gray_scale=True )
    equ_img_resize = cv2.resize(equ_img,(200,200))
    images = kimg.img_to_array(equ_img_resize)
    gray_img=np.expand_dims(images,axis=0)
    ans = model.predict(gray_img)
    ans = np.argmax(ans)
    print("Model Prediction:",ans)
    return ans

def main(imagefilename):
    model = model_architecture()
    predictions = mymodel(imagefilename,model)
    print(predictions)
    return predictions


In [55]:
main('p1.jpg')

Model Prediction: 0
0


0

In [ ]:
def read_this(image_file, gray_scale=False):
    image_src = cv2.imread(image_file)
    if gray_scale:
        image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
    else:
        image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
    return image_src

def enhance_contrast(image_matrix, bins=256):
    image_flattened = image_matrix.flatten()
    image_hist = np.zeros(bins)

    # frequency count of each pixel
    for pix in image_matrix:
        image_hist[pix] += 1

    # cummulative sum
    cum_sum = np.cumsum(image_hist)
    norm = (cum_sum - cum_sum.min()) * 255
    # normalization of the pixel values
    n_ = cum_sum.max() - cum_sum.min()
    uniform_norm = norm / n_
    uniform_norm = uniform_norm.astype(np.uint8)

    # flat histogram
    image_eq = uniform_norm[image_flattened]
    # reshaping the flattened matrix to its original shape
    image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)

    return image_eq

def equalize_this(image_src, with_plot=False, gray_scale=False, bins=256):
#     image_src = read_this(image_file=image_file, gray_scale=gray_scale)
    if not gray_scale:
        r_image = image_src[:, :, 0]
        g_image = image_src[:, :, 1]
        b_image = image_src[:, :, 2]

        r_image_eq = enhance_contrast(image_matrix=r_image)
        g_image_eq = enhance_contrast(image_matrix=g_image)
        b_image_eq = enhance_contrast(image_matrix=b_image)

        image_eq = np.dstack(tup=(r_image_eq, g_image_eq, b_image_eq))
        cmap_val = None
        image_eq = image_eq.astype(np.uint8)
        #         cv2.imwrite(img, image_eq)
        #io.imsave(img, image_eq)             #save the equilised image by replacing the original image.

    else:
        image_eq = enhance_contrast(image_matrix=image_src)
        cmap_val = 'gray'
        #         cv2.imwrite(img, image_eq)
        #io.imsave(img, image_eq)             #save the equilised image by replacing the original image.
    return image_eq

